In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from functools import reduce
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 150)
def percentchange(x, y):
    try:
        return ((x - y)*100/y)
    except ZeroDivisionError:
        return 0
def realchange(x, y):
    return x-y
def percent(x, y):
    return (x/y)*100

## This notebook outlines the high level industry data formatting for the Comprehensive Plans with place level primary geographies. There is another document for the information that is pulled for all relevant geographies.

This retrieves historical and current annual occupational employment disaggregated by high level occupation group.

To retrieve data:
+ Log into JobsEQ and navigate to the "Data Explorer" function  
+ Select the "Occupation" dataset, annual  
+ Select the following dimensions: Region, Industry, Year/Quarter, then select "Employment" as the value  
+ Under "Region" select "GNRC Region" and sub "Census Place", then delete "GNRC Region" as that is covered from the Woods & Poole data 
+ For "Year/Quarter" select as many as possible  
+ Under "Occupation" expand "Total-All Occupations" and select the combination of 2 digit SOCS codes and built groups in Jobs EQ as described below, leaving Total as well  

Occupations to Select:  
(SOC Codes)  
+ Total - All Occupations  
+ Healthcare Practitioners and Technical Occupations (29)  
+ Healthcare Support Occupations (31)  
+ Protective Service Occupations (33)  
+ Production Occupations (51)  
+ Transportation and Material Moving Occupations (53)  
(Custom Created Groups)  
+ Business & Financial (11: Management & 13: Business & Financial Operations)  
+ Computer, Mathematical, & Engineering (15: Computer & Mathematical, 17: Architecture & Engineering, & 19: Life, Physical, & Social Science)  
+ Education, Legal, Social, Arts, & Media (21: Community & Social Service, 23: Legal, 25: Educational Instruction & Library, & 27: Art, Design, Entertainment, Sports, & Media)  
+ Food, Property, & Personal Services (35: Food Preparation & Serving, 37: Building & Grounds Cleaning & Maintenance, & 39: Personal Care & Service)  
+ Sales & Office Support (41: Sales & Related & 43: Office & Administrative Support)  
+ Natural Resources, Construction, & Maintenance (45: Farming Fishing & Forestry, 47: Construction & Extraction, & 49: Installation Maintenance & Repair)

This may exceed the drafting size, in which case you'll have to queu for download. Once it is completed, open and delete the top row that says "Industry Data", and delete the third row that repeats "Employment". Rename the FIPS column to GEO_ID, "Time" to "Year", and "Region" to "NAME". Use text to columns to change the GEO_ID column to text. Save in the "Data Downloads" folder as "JobsEQIndustryEmployment_Place.csv"

In [2]:
data = pd.read_csv('../../Data Downloads/JobsEQOccupationalEmployment_timeseries_allgeos.csv')

In [3]:
data = data.loc[(data['Year'] != 2012)&(data['Year'] != 2022)]

In [4]:
years = [2013, 2014, 2015, 2016, 2017, 2019, 2019, 2020, 2021]
geos = list(data.NAME.unique())

In [5]:
data = data.set_index(['Quarter', 'Year', 'NAME']).transpose()

In [6]:
cols = data.columns

In [7]:
for year in years:
    for geo in geos:
        thelist = [data[('Q1', year, '{}'.format(geo))], data[('Q2', year, '{}'.format(geo))], data[('Q3', year, '{}'.format(geo))], 
                  data[('Q4', year, '{}'.format(geo))]]
        data[('All', year, '{}'.format(geo))] = (sum(thelist))/4

C:\Users\jmccall\AppData\Local\Temp\ipykernel_12308\2707240853.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[('All', year, '{}'.format(geo))] = (sum(thelist))/4


In [8]:
data = data.drop(columns = cols)

In [9]:
data.head()

Quarter                                                      All                                                                                    \
Year                                                        2013                                                                                     
NAME                                                         USA   Tennessee GNRC Region Adams city, TN Ashland City town, TN Belle Meade city, TN   
Total - All Occupations                             1.442553e+08  2942359.25   929693.75           54.5               3894.00              1166.00   
Healthcare Practitioners and Technical Occupati...  7.602078e+06   175451.00    56008.75            1.0                 56.00                27.25   
Healthcare Support Occupations (31-0000)            5.792251e+06   109286.00    33990.50            3.0                 40.75                14.75   
Protective Service Occupations (33-0000)            3.211846e+06    63871.25    21036.00            1.0                104.25                21.00   
Production Occupations (51-0000)                    9.241163e+06   250650.75    66504.75            1.0                959.00                21.50   

Quarter                                                                                                                                      \
Year                                                                                                                                          
NAME                                               Berry Hill city, TN Bethpage CDP, TN Bransford CDP, TN Brentwood city, TN Burns town, TN   
Total - All Occupations                                       17785.00             52.0              10.0            40637.0         233.75   
Healthcare Practitioners and Technical Occupati...              552.50              1.0               0.0             2021.0           7.25   
Healthcare Support Occupations (31-0000)                        326.00              0.0               0.0             2137.0           6.00   
Protective Service Occupations (33-0000)                       1970.75              1.0               0.0              285.0           5.00   
Production Occupations (51-0000)                                553.00              1.0               1.0              765.0           3.00   

Quarter                                                                                                                                 \
Year                                                                                                                                     
NAME                                               Castalian Springs CDP, TN Cedar Hill city, TN Charlotte town, TN Christiana CDP, TN   
Total - All Occupations                                                28.25                10.0             799.25             282.00   
Healthcare Practitioners and Technical Occupati...                      0.25                 0.0              19.25               5.25   
Healthcare Support Occupations (31-0000)                                0.00                 0.0               6.00               1.00   
Protective Service Occupations (33-0000)                                0.00                 0.0             122.50               5.00   
Production Occupations (51-0000)                                        2.00                 0.0              18.50               2.00   

Quarter                                                                                                                           \
Year                                                                                                                               
NAME                                               Clarksville city, TN Columbia city, TN Coopertown town, TN Cottontown CDP, TN   
Total - All Occupations                                        43716.50          19794.00              452.50               15.5   
Healthcare Practition

In [10]:
data = data.transpose()

In [11]:
data = data.reset_index(drop = False).drop(columns = 'Quarter')
data.head()

,Year,NAME,Total - All Occupations,Healthcare Practitioners and Technical Occupations (29-0000),Healthcare Support Occupations (31-0000),Protective Service Occupations (33-0000),Production Occupations (51-0000),Transportation and Material Moving Occupations (53-0000),Business & Financial,"Computer, Mathematical, & Engineering","Education, Legal, Social, Arts, & Media","Food, Property, & Personal Services",Sales & Office Support,"Natural Resources, Construction, & Maintenance"
0,2013,USA,1.442553e+08,7602077.75,5792251.25,3211845.75,9241163.25,11371218.25,15334089.00,6758341.75,14974925.50,20829475.50,36038631.50,13101245.25
1,2013,Tennessee,2.942359e+06,175451.00,109286.00,63871.25,250650.75,296318.25,284072.75,103408.50,271243.75,413931.00,721557.75,252568.75
2,2013,GNRC Region,9.296938e+05,56008.75,33990.50,21036.00,66504.75,80565.75,94931.25,33881.75,94658.00,132455.25,240654.00,75008.50
3,2013,"Adams city, TN",5.450000e+01,1.00,3.00,1.00,1.00,6.25,4.50,1.00,6.75,5.75,21.00,3.75
4,2013,"Ashland City town, TN",3.894000e+03,56.00,40.75,104.25,959.00,563.25,370.50,168.50,208.00,371.75,667.00,385.50


In [12]:
data.head()

,Year,NAME,Total - All Occupations,Healthcare Practitioners and Technical Occupations (29-0000),Healthcare Support Occupations (31-0000),Protective Service Occupations (33-0000),Production Occupations (51-0000),Transportation and Material Moving Occupations (53-0000),Business & Financial,"Computer, Mathematical, & Engineering","Education, Legal, Social, Arts, & Media","Food, Property, & Personal Services",Sales & Office Support,"Natural Resources, Construction, & Maintenance"
0,2013,USA,1.442553e+08,7602077.75,5792251.25,3211845.75,9241163.25,11371218.25,15334089.00,6758341.75,14974925.50,20829475.50,36038631.50,13101245.25
1,2013,Tennessee,2.942359e+06,175451.00,109286.00,63871.25,250650.75,296318.25,284072.75,103408.50,271243.75,413931.00,721557.75,252568.75
2,2013,GNRC Region,9.296938e+05,56008.75,33990.50,21036.00,66504.75,80565.75,94931.25,33881.75,94658.00,132455.25,240654.00,75008.50
3,2013,"Adams city, TN",5.450000e+01,1.00,3.00,1.00,1.00,6.25,4.50,1.00,6.75,5.75,21.00,3.75
4,2013,"Ashland City town, TN",3.894000e+03,56.00,40.75,104.25,959.00,563.25,370.50,168.50,208.00,371.75,667.00,385.50


In [13]:
#resum so that percentages work - data caveat that they would not sum perfectly due to rounding
thelist = [data['Healthcare Practitioners and Technical Occupations (29-0000)'], data['Healthcare Support Occupations (31-0000)'], 
           data['Protective Service Occupations (33-0000)'], data['Production Occupations (51-0000)'], 
           data['Transportation and Material Moving Occupations (53-0000)'], data['Business & Financial'],
           data['Computer, Mathematical, & Engineering'], data['Education, Legal, Social, Arts, & Media'], 
           data['Food, Property, & Personal Services'], data['Sales & Office Support'], data['Natural Resources, Construction, & Maintenance']]
data['Total - All Occupations'] = sum(thelist)

In [14]:
data = data.rename(columns = {'Healthcare Practitioners and Technical Occupations (29-0000)': 'Healthcare Practitioners & Technical',
                              'Healthcare Support Occupations (31-0000)': 'Healthcare Support', 
                              'Protective Service Occupations (33-0000)': 'Protective Services',
                              'Production Occupations (51-0000)': 'Production', 
                              'Transportation and Material Moving Occupations (53-0000)': 'Transportation & Material Moving', 
                             'Total - All Occupations': 'Total'})

In [15]:
data.head()

,Year,NAME,Total,Healthcare Practitioners & Technical,Healthcare Support,Protective Services,Production,Transportation & Material Moving,Business & Financial,"Computer, Mathematical, & Engineering","Education, Legal, Social, Arts, & Media","Food, Property, & Personal Services",Sales & Office Support,"Natural Resources, Construction, & Maintenance"
0,2013,USA,1.442553e+08,7602077.75,5792251.25,3211845.75,9241163.25,11371218.25,15334089.00,6758341.75,14974925.50,20829475.50,36038631.50,13101245.25
1,2013,Tennessee,2.942360e+06,175451.00,109286.00,63871.25,250650.75,296318.25,284072.75,103408.50,271243.75,413931.00,721557.75,252568.75
2,2013,GNRC Region,9.296945e+05,56008.75,33990.50,21036.00,66504.75,80565.75,94931.25,33881.75,94658.00,132455.25,240654.00,75008.50
3,2013,"Adams city, TN",5.500000e+01,1.00,3.00,1.00,1.00,6.25,4.50,1.00,6.75,5.75,21.00,3.75
4,2013,"Ashland City town, TN",3.894500e+03,56.00,40.75,104.25,959.00,563.25,370.50,168.50,208.00,371.75,667.00,385.50


In [16]:
cols = data.columns

In [17]:
cols = data.columns
thing = data.melt(var_name = 'Occupation', id_vars = ['NAME', 'Year'], value_vars = cols, value_name = 'Employment')
thing.head()

,NAME,Year,Occupation,Employment
0,USA,2013,Total,1.442553e+08
1,Tennessee,2013,Total,2.942360e+06
2,GNRC Region,2013,Total,9.296945e+05
3,"Adams city, TN",2013,Total,5.500000e+01
4,"Ashland City town, TN",2013,Total,3.894500e+03


In [18]:
percents = data

In [19]:
percents.head()

,Year,NAME,Total,Healthcare Practitioners & Technical,Healthcare Support,Protective Services,Production,Transportation & Material Moving,Business & Financial,"Computer, Mathematical, & Engineering","Education, Legal, Social, Arts, & Media","Food, Property, & Personal Services",Sales & Office Support,"Natural Resources, Construction, & Maintenance"
0,2013,USA,1.442553e+08,7602077.75,5792251.25,3211845.75,9241163.25,11371218.25,15334089.00,6758341.75,14974925.50,20829475.50,36038631.50,13101245.25
1,2013,Tennessee,2.942360e+06,175451.00,109286.00,63871.25,250650.75,296318.25,284072.75,103408.50,271243.75,413931.00,721557.75,252568.75
2,2013,GNRC Region,9.296945e+05,56008.75,33990.50,21036.00,66504.75,80565.75,94931.25,33881.75,94658.00,132455.25,240654.00,75008.50
3,2013,"Adams city, TN",5.500000e+01,1.00,3.00,1.00,1.00,6.25,4.50,1.00,6.75,5.75,21.00,3.75
4,2013,"Ashland City town, TN",3.894500e+03,56.00,40.75,104.25,959.00,563.25,370.50,168.50,208.00,371.75,667.00,385.50


In [20]:
#first we will aggregate some of these industries to match our other industry data
percents['Healthcare Practitioners & Technical'] = percent(percents['Healthcare Practitioners & Technical'], percents['Total'])
percents['Healthcare Support'] = percent(percents['Healthcare Support'], percents['Total'])
percents['Protective Services'] = percent(percents['Protective Services'], percents['Total'])
percents['Production'] = percent(percents['Production'], percents['Total'])
percents['Transportation & Material Moving'] = percent(percents['Transportation & Material Moving'], percents['Total'])
percents['Business & Financial'] = percent(percents['Business & Financial'], percents['Total'])
percents['Computer, Mathematical, & Engineering'] = percent(percents['Computer, Mathematical, & Engineering'], percents['Total'])
percents['Education, Legal, Social, Arts, & Media'] = percent(percents['Education, Legal, Social, Arts, & Media'], percents['Total'])
percents['Food, Property, & Personal Services'] = percent(percents['Food, Property, & Personal Services'], percents['Total'])
percents['Sales & Office Support'] = percent(percents['Sales & Office Support'], percents['Total'])
percents['Natural Resources, Construction, & Maintenance'] = percent(percents['Natural Resources, Construction, & Maintenance'], percents['Total'])

In [21]:
thing.head()

,NAME,Year,Occupation,Employment
0,USA,2013,Total,1.442553e+08
1,Tennessee,2013,Total,2.942360e+06
2,GNRC Region,2013,Total,9.296945e+05
3,"Adams city, TN",2013,Total,5.500000e+01
4,"Ashland City town, TN",2013,Total,3.894500e+03


In [22]:
cols = percents.columns
percents = percents.reset_index(drop = False).melt(var_name = 'Occupation', id_vars = ['NAME', 'Year'], value_vars = cols, value_name = 'Percent of Employment')
percents.head()

,NAME,Year,Occupation,Percent of Employment
0,USA,2013,Total,1.442553e+08
1,Tennessee,2013,Total,2.942360e+06
2,GNRC Region,2013,Total,9.296945e+05
3,"Adams city, TN",2013,Total,5.500000e+01
4,"Ashland City town, TN",2013,Total,3.894500e+03


In [23]:
percents = percents.set_index('Occupation').transpose()
percents = percents.drop(columns = 'Total')
percents = percents.transpose().reset_index(drop = False)

In [24]:
df = thing.merge(percents, on = ['Occupation', 'Year', 'NAME'], how = 'outer')
df.head(20)

,NAME,Year,Occupation,Employment,Percent of Employment
0,USA,2013,Total,1.442553e+08,NaN
1,Tennessee,2013,Total,2.942360e+06,NaN
2,GNRC Region,2013,Total,9.296945e+05,NaN
3,"Adams city, TN",2013,Total,5.500000e+01,NaN
4,"Ashland City town, TN",2013,Total,3.894500e+03,NaN
5,"Belle Meade city, TN",2013,Total,1.165750e+03,NaN
6,"Berry Hill city, TN",2013,Total,1.778375e+04,NaN
7,"Bethpage CDP, TN",2013,Total,5.200000e+01,NaN
8,"Bransford CDP, TN",2013,Total,8.500000e+00,NaN
9,"Brentwood city, TN",2013,Total,4.063800e+04,NaN


In [25]:
df = df.set_index('NAME').transpose()

In [26]:
df = df.rename(columns = {'Adams city, TN': 'Adams', 'Ashland City town, TN': 'Ashland City', 'Belle Meade city, TN': 'Belle Meade',
                                    'Berry Hill city, TN': 'Berry Hill', 'Brentwood city, TN': 'Brentwood', 'Burns town, TN': 'Burns',
                                    'Cedar Hill city, TN': 'Cedar Hill', 'Charlotte town, TN': 'Charlotte',
                                    'Cheatham County, Tennessee': 'Cheatham County', 'Clarksville city, TN': 'Clarksville',
                                    'Columbia city, TN': 'Columbia', 'Coopertown town, TN': 'Coopertown',
                                    'Cross Plains city, TN': 'Cross Plains', 'Cumberland City town, TN': 'Cumberland City',
                                    'Davidson County, Tennessee': 'Davidson County', 'Dickson city, TN': 'Dickson',
                                    'Dickson County, Tennessee': 'Dickson County', 'Dover city, TN': 'Dover',
                                    'Eagleville city, TN': 'Eagleville', 'Erin city, TN': 'Erin', 'Fairview city, TN': 'Fairview',
                                    'Forest Hills city, TN': 'Forest Hills', 'Franklin city, TN': 'Franklin', 'Gallatin city, TN': 'Gallatin',
                                    'Goodlettsville city, TN': 'Goodlettsville', 'Greenbrier town, TN': 'Greenbrier',
                                    'Hartsville/Trousdale County, TN': 'Hartsville/Trousdale', 'Hendersonville city, TN': 'Hendersonville',
                                    'Houston County, Tennessee': 'Houston County', 'Humphreys County, Tennessee': 'Humphreys County',
                                    'Kingston Springs town, TN': 'Kingston Springs', 'La Vergne city, TN': 'La Vergne',
                                    'Lebanon city, TN': 'Lebanon', 'McEwen city, TN': 'McEwen','Maury County, Tennessee': 'Maury County',
                                    'Millersville city, TN': 'Millersville', 'Mitchellville city, TN': 'Mitchellville',
                                    'Montgomery County, Tennessee': 'Montgomery County', 'Mount Juliet city, TN': 'Mount Juliet',
                                    'Mount Pleasant city, TN': 'Mount Pleasant', 'Murfreesboro city, TN': 'Murfreesboro',
                                    'Nashville-Davidson metropolitan government (balance), TN': 'Nashville', 
                                    'New Johnsonville city, TN': 'New Johnsonville', 'Nolensville town, TN': 'Nolensville',
                                    'Oak Hill city, TN': 'Oak Hill', 'Pegram town, TN': 'Pegram', 'Pleasant View city, TN': 'Pleasant View',
                                    'Portland city, TN': 'Portland', 'Ridgetop city, TN': 'Ridgetop','Robertson County, Tennessee': 'Robertson County',
                                    'Rutherford County, Tennessee': 'Rutherford County', 'Slayden town, TN': 'Slayden', 'Smyrna town, TN': 'Smyrna',
                                    'Spring Hill city, TN': 'Spring Hill', 'Springfield city, TN': 'Springfield',
                                    'Stewart County, Tennessee': 'Stewart County', 'Sumner County, Tennessee': 'Sumner County',
                                    'Tennessee Ridge town, TN': 'Tennessee Ridge', "Thompson's Station town, TN": "Thompson's Station",
                                    'Trousdale County, Tennessee': 'Trousdale County', 'Vanleer town, TN': 'Vanleer', 'Watertown city, TN': 'Watertown',
                                    'Waverly city, TN': 'Waverly', 'Westmoreland town, TN': 'Westmoreland', 'White Bluff town, TN': 'White Bluff',
                                    'White House city, TN': 'White House', 'Williamson County, Tennessee': 'Williamson County',
                                    'Wilson County, Tennessee': 'Wilson County'})
df = df.transpose()
df = df.reset_index()

In [27]:
df.tail(200)

,NAME,Year,Occupation,Employment,Percent of Employment
8344,"Walterhill CDP, TN",2019,"Natural Resources, Construction, & Maintenance",56.5,29.581152
8345,Watertown,2019,"Natural Resources, Construction, & Maintenance",57.5,12.412304
8346,Waverly,2019,"Natural Resources, Construction, & Maintenance",208.5,9.264608
8347,Westmoreland,2019,"Natural Resources, Construction, & Maintenance",40.0,4.599023
8348,White Bluff,2019,"Natural Resources, Construction, & Maintenance",113.75,12.333966
8349,White House,2019,"Natural Resources, Construction, & Maintenance",387.0,8.065861
8350,Cheatham County,2019,"Natural Resources, Construction, & Maintenance",1314.0,12.757901
8351,Davidson County,2019,"Natural Resources, Construction, & Maintenance",41710.25,7.726123
8352,Dickson County,2019,"Natural Resources, Construction, & Maintenance",2100.25,10.433821
8353,Houston County,2019,"Natural Resources, Construction, & Maintenance",219.25,11.745011


In [131]:
df.to_csv('../../Outputs/JOBSEQ_OCCUPATIONALEMP_ALLGEOS_HISTORICALANDCURRENT.csv', index = False)